In [1]:
from datetime import datetime

from libs import (
    test_acceleration,
    print_estimation,
    bootstrap_growth_parameters,
    sliding_window_analysis,
)
from model_data import model_data

print("Loaded libraries")

Loaded libraries


In [2]:
# Construct observed_models from model_data
# Format: (model_name, release_date, task_length_50%, task_length_80%) in hours
observed_models = [
    (model['name'], model['launch_date'], model['performance_50p'], model['performance_80p'])
    for model in model_data.values()
    if model['performance_50p'] is not None  # Exclude models without data
]

print("=== 50% Reliability ===")
print_estimation(observed_models)
print_estimation(observed_models[1:])
print_estimation(observed_models[3:])
print_estimation(observed_models[5:])

print_estimation(observed_models[1:-1])
print_estimation(observed_models[3:-1])
print_estimation(observed_models[5:-1])
print_estimation(observed_models[5:-1])

print("\n=== 80% Reliability ===")
print_estimation(observed_models, "80%")
print_estimation(observed_models[1:], "80%")
print_estimation(observed_models[3:], "80%")
print_estimation(observed_models[5:], "80%")

print_estimation(observed_models[1:-1], "80%")
print_estimation(observed_models[3:-1], "80%")
print_estimation(observed_models[5:-1], "80%")
print_estimation(observed_models[5:-1], "80%")

=== 50% Reliability ===
GPT‑2 to Claude 4.5 Opus (50%): (310, 0.913)
GPT-3 to Claude 4.5 Opus (50%): (297, 0.9)
GPT-4 to Claude 4.5 Opus (50%): (235, 0.9)
Claude 3 Opus to Claude 4.5 Opus (50%): (118, 1.0)
GPT-3 to GPT5.1-Codex-Max (50%): (297, 0.9)
GPT-4 to GPT5.1-Codex-Max (50%): (237, 0.9)
Claude 3 Opus to GPT5.1-Codex-Max (50%): (119, 1.0)
Claude 3 Opus to GPT5.1-Codex-Max (50%): (119, 1.0)

=== 80% Reliability ===
GPT‑2 to Claude 4.5 Opus (80%): (248, 0.934)
GPT-3 to Claude 4.5 Opus (80%): (297, 0.9)
GPT-4 to Claude 4.5 Opus (80%): (217, 0.9)
Claude 3 Opus to Claude 4.5 Opus (80%): (121, 0.9)
GPT-3 to GPT5.1-Codex-Max (80%): (302, 0.9)
GPT-4 to GPT5.1-Codex-Max (80%): (238, 0.9)
Claude 3 Opus to GPT5.1-Codex-Max (80%): (111, 1.0)
Claude 3 Opus to GPT5.1-Codex-Max (80%): (111, 1.0)


In [3]:
print("=== Bootstrap Analysis ===")
current_date = datetime.today()
print(f"Current date: {current_date.strftime('%Y-%m-%d')}")

for rel in ["50%", "80%"]:
    print(f"\n{rel} Reliability:")
    full_results = bootstrap_growth_parameters(
        observed_models, reliability_level=rel, current_date=current_date
    )
    print(f"Full dataset: {full_results['median']} (95% CI: {full_results['ci_95']})")
    recent_models = [m for m in observed_models if m[1] >= datetime(2024, 1, 1)]
    recent_results = bootstrap_growth_parameters(
        recent_models, reliability_level=rel, current_date=current_date
    )
    print(
        f"2024+ models: {recent_results['median']} (95% CI: {recent_results['ci_95']})"
    )

df = sliding_window_analysis(observed_models)
print("\n=== Parameter stability by time window ===")
print(df.groupby("window")[["doubling_time", "acceleration"]].agg(["mean", "std"]))

=== Bootstrap Analysis ===
Current date: 2025-12-20

50% Reliability:
Full dataset: (295, 0.908) (95% CI: {'doubling_time': (112, 333), 'acceleration': (0.9, 1.0)})
2024+ models: (122, 1.0) (95% CI: {'doubling_time': (113, 175), 'acceleration': (0.9, 1.0)})

80% Reliability:
Full dataset: (265, 0.9) (95% CI: {'doubling_time': (124, 304), 'acceleration': (0.9, 1.0)})
2024+ models: (120, 0.9) (95% CI: {'doubling_time': (101, 147), 'acceleration': (0.9, 1.0)})

=== Parameter stability by time window ===
       doubling_time            acceleration          
                mean        std         mean       std
window                                                
6         136.117647  90.728222     0.958765  0.050680
8         154.600000  79.099214     0.960000  0.050709
10        170.769231  71.228686     0.953462  0.051534
12        181.909091  72.980072     0.928636  0.046048


In [4]:
test_acceleration(
    start_task_length=model_data['gpt2']['performance_50p'],
    agi_task_length=167,
    initial_doubling_time=260,
    acceleration=0.95,
    start_date=model_data['gpt2']['launch_date'].strftime('%Y-%m-%d'),
)

Step |    Date    |    Day |       Task | τ (d)
-----------------------------------------------
   0 | 2019‑02‑14 |      0 |       2sec | 260.0
   1 | 2019‑11‑01 |    260 |       4sec | 247.0
   2 | 2020‑07‑05 |    507 |       8sec | 234.6
   3 | 2021‑02‑24 |    741 |      16sec | 222.9
   4 | 2021‑10‑05 |    964 |      32sec | 211.8
   5 | 2022‑05‑05 |   1176 |    1.07min | 201.2
   6 | 2022‑11‑22 |   1377 |    2.13min | 191.1
   7 | 2023‑06‑01 |   1568 |    4.27min | 181.6
   8 | 2023‑11‑30 |   1750 |    8.53min | 172.5
   9 | 2024‑05‑20 |   1922 |   17.07min | 163.9
  10 | 2024‑10‑31 |   2086 |   34.13min | 155.7
  11 | 2025‑04‑05 |   2242 |     1.14hr | 147.9
  12 | 2025‑08‑31 |   2390 |     2.28hr | 140.5
  13 | 2026‑01‑18 |   2530 |     4.55hr | 133.5
  14 | 2026‑06‑01 |   2664 |     9.10hr | 126.8
  15 | 2026‑10‑05 |   2790 |    18.20hr | 120.5
  16 | 2027‑02‑03 |   2911 |    36.41hr | 114.4
  17 | 2027‑05‑28 |   3025 |    72.82hr | 108.7
  18 | 2027‑09‑14 |   3134 |   145.64hr 

In [ ]:
test_acceleration(
    start_task_length=model_data['gpt2']['performance_50p'],
    agi_task_length=167,
    initial_doubling_time=260,
    acceleration=0.95,
    start_date=model_data['gpt2']['launch_date'].strftime('%Y-%m-%d'),
)